In [1]:
#%pip install cython ipywidgets nltk

#import nltk
#nltk.download('punkt')

In [1]:
%load_ext autoreload
%autoreload 2

import os
import string
import pandas as pd
import numpy as np
import torch
import json
import gc

from collections import defaultdict

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from tqdm.auto import tqdm

from loader.dataset_coco import COCODataset
from loader.images import ImageS3
from loader.model import ModelS3
from loader.vocab import construct_vocab

from commons.utils import embedding_matrix, tensor_to_word_fn
from models.seq2seq import Seq2SeqAttention
from eval.metrics import bleu, cider, rouge, spice, meteor, bleu_score_fn


from IPython.core.display import HTML

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
train = COCODataset(dtype='train', ret_type='tensor', copy_img_to_mem=False, device=device, partial=100) #
val = COCODataset(dtype='val', ret_type='tensor', copy_img_to_mem=False, device=device, partial=50) #

loading annotations into memory...
Done (t=1.27s)
creating index...
index created!
tokenizing caption... done!!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
tokenizing caption... done!!


In [3]:
vocabulary = construct_vocab(train.df, val.df)
train.vocabulary = vocabulary
val.vocabulary = vocabulary

In [4]:
val_eval = COCODataset(dtype='val', ret_type='corpus', copy_img_to_mem=False, vocabulary=vocabulary, device=device, partial=50) #

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
tokenizing caption... done!!


In [8]:
test = COCODataset(dtype='test', ret_type='corpus', copy_img_to_mem=False, vocabulary=vocabulary, device=device, partial=10) #

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
no caption found... done!!
